In [13]:
# Import  libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report

# Load the Digits dataset
data_set = load_digits()

# Prepare the data
X = data_set.data
Y = data_set.target

# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=72)
pca = PCA(n_components=50)  
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
model_qda = QuadraticDiscriminantAnalysis()
best_reg_param = None
best_accuracy = 0
for reg_param in np.logspace(-3, 0, 4):  
    model_qda.set_params(reg_param=reg_param)
    model_qda.fit(X_train_pca, Y_train)
    accuracy = accuracy_score(Y_test, model_qda.predict(X_test_pca))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_reg_param = reg_param
print(f"Best reg_param found: {best_reg_param}")
model_qda.set_params(reg_param=best_reg_param)
model_qda.fit(X_train_pca, Y_train)
pred_qda = model_qda.predict(X_test_pca)
bal_acc_qda = balanced_accuracy_score(Y_test, pred_qda)
print(f"Accuracy: {accuracy_score(Y_test, pred_qda) * 100:.2f}%")
print(f"Balanced Accuracy: {bal_acc_qda * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(Y_test, pred_qda))
print("Classification Report:")
print(classification_report(Y_test, pred_qda))
roc_qda = None  


Best reg_param found: 0.1
Accuracy: 98.61%
Balanced Accuracy: 98.54%
Confusion Matrix:
[[40  0  0  0  0  0  0  0  0  0]
 [ 0 32  0  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  1  0  0  0  0]
 [ 0  0  0  0 30  0  0  0  1  1]
 [ 0  0  0  0  0 36  0  0  0  0]
 [ 0  0  0  0  0  1 38  0  0  0]
 [ 0  0  0  0  0  0  0 31  0  0]
 [ 0  0  0  0  0  0  0  0 38  0]
 [ 0  0  0  0  0  0  0  1  0 40]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00        41
           3       1.00      0.97      0.98        30
           4       1.00      0.94      0.97        32
           5       0.95      1.00      0.97        36
           6       1.00      0.97      0.99        39
           7       0.97      1.00      0.98        31
           8       0.97      1.00      0.99        38
           9      